<h1 style="text-align: center; font-size: 36px; color: teal">NFLAI Report Generator</h1>
<div style="text-align: center;">
    <h3 style="color: black;">Created by Tyler Durette</h3>
    <img src="https://media.bleacherreport.com/image/upload/c_crop,h_0.90,w_0.90,x_0.01,y_0.05/w_650/v1693966577/wy9rjhloyplkvjq7ljgz.jpg" alt="NFL AI Logo" width="800"/>
    <hr style="border: 2px solid purple; width: 65%;">
</div>


## INCLUDE

1. Injury report daily appearances
2. Travel distance
3. Days rest
4. Last 5 game outcomes
5. Player hot and cold TD streaks

---

In [1]:
import pandas as pd
import sqlite3
import os
import requests

In [2]:
!rm -rf data
!cp -r ../Scrapers/data .
!rm data/games.csv
!rm data/player_stats.csv
!rm data/rosters.csv
!cp ../Scrapers/nfl.db data/
!cp ../Scrapers/PlayerStats.csv data/
!cp ../Scrapers/Rosters.csv data/
!cp ../Scrapers/Games.csv data/
!cp ../Scrapers/Teams.csv data/
file_path = 'data/PlayerStats.csv'
data = pd.read_csv(file_path)
filtered_data = data[data['season'].between(2020, 2024)]
filtered_file_path = 'data/PlayerStats.csv'
filtered_data.to_csv(filtered_file_path, index=False)
file_path_rosters = 'data/Rosters.csv'
rosters_data = pd.read_csv(file_path_rosters)
filtered_rosters_data = rosters_data[rosters_data['season'].between(2020, 2024)]
filtered_rosters_file_path_new = 'data/Rosters.csv'
filtered_rosters_data.to_csv(filtered_rosters_file_path_new, index=False)
file_path = 'data/Games.csv'
games_df = pd.read_csv(file_path)
unplayed_games_df = games_df[games_df['home_score'].isna() & games_df['away_score'].isna()]
unplayed_file_path = 'data/UnplayedGames.csv'
unplayed_games_df.to_csv(unplayed_file_path, index=False)
played_games_df = games_df.dropna(subset=['home_score', 'away_score'])
played_file_path = 'data/Games.csv'
played_games_df.to_csv(played_file_path, index=False)
file_path = 'data/all_passing_rushing_receiving.csv'  # Adjust this path as needed
df = pd.read_csv(file_path)
row_count_before = df.shape[0]
print(f"Row count before removing duplicates: {row_count_before}")
df_cleaned = df.drop_duplicates()
row_count_after = df_cleaned.shape[0]
print(f"Row count after removing duplicates: {row_count_after}")
df_cleaned.to_csv('data/all_passing_rushing_receiving.csv', index=False)

Row count before removing duplicates: 35568
Row count after removing duplicates: 6961


In [ ]:
# Downloading logos

os.makedirs('images', exist_ok=True)
team_logos = {
    "https://cdn.fastassets.io/images/team-logos/New-Orleans-Saints.png": "NO",
    "https://cdn.fastassets.io/images/team-logos/Atlanta-Falcons.png": "ATL",
    "https://cdn.fastassets.io/images/team-logos/Pittsburgh-Steelers.png": "PIT",
    "https://cdn.fastassets.io/images/team-logos/Indianapolis-Colts.png": "IND",
    "https://cdn.fastassets.io/images/team-logos/Jacksonville-Jaguars.png": "JAX",
    "https://cdn.fastassets.io/images/team-logos/Houston-Texans.png": "HOU",
    "https://cdn.fastassets.io/images/team-logos/Minnesota-Vikings.png": "MIN",
    "https://cdn.fastassets.io/images/team-logos/Green-Bay-Packers.png": "GB",
    "https://cdn.fastassets.io/images/team-logos/Cincinnati-Bengals.png": "CIN",
    "https://cdn.fastassets.io/images/team-logos/Carolina-Panthers.png": "CAR",
    "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/lar.png": "LAR",
    "https://cdn.fastassets.io/images/team-logos/Chicago-Bears.png": "CHI",
    "https://cdn.fastassets.io/images/team-logos/Denver-Broncos.png": "DEN",
    "https://cdn.fastassets.io/images/team-logos/Tampa-Bay-Buccaneers.png": "TB",
    "https://cdn.fastassets.io/images/team-logos/Seattle-Seahawks.png": "SEA",
    "https://cdn.fastassets.io/images/team-logos/San-Francisco-49ers.png": "SF",
    "https://cdn.fastassets.io/images/team-logos/Arizona-Cardinals.png": "ARI",
    "https://cdn.fastassets.io/images/team-logos/Dallas-Cowboys.png": "DAL",
    "https://cdn.fastassets.io/images/team-logos/Philadelphia-Eagles.png": "PHI",
    "https://cdn.fastassets.io/images/team-logos/Washington-Redskins.png": "WAS",  # Corrected
    "https://cdn.fastassets.io/images/team-logos/New-York-Giants.png": "NYG",
    "https://cdn.fastassets.io/images/team-logos/Kansas-City-Chiefs.png": "KC",
    "https://s.yimg.com/cv/apiv2/default/nfl/20200908/500x500/raiders_wbg.png": "LVR",  # Corrected
    "https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/lac.png": "LAC",  # Corrected
    "https://cdn.fastassets.io/images/team-logos/Buffalo-Bills.png": "BUF",
    "https://cdn.fastassets.io/images/team-logos/Miami-Dolphins.png": "MIA",
    "https://cdn.fastassets.io/images/team-logos/New-York-Jets.png": "NYJ",
    "https://cdn.fastassets.io/images/team-logos/New-England-Patriots.png": "NE",
    "https://cdn.fastassets.io/images/team-logos/Cleveland-Browns.png": "CLE",
    "https://cdn.fastassets.io/images/team-logos/Baltimore-Ravens.png": "BAL",
    "https://cdn.fastassets.io/images/team-logos/Detroit-Lions.png": "DET",
    "https://cdn.fastassets.io/images/team-logos/Tennessee-Titans.png": "TEN"
}
for url, abbreviation in team_logos.items():
    try:
        img_data = requests.get(url).content  # Download the image content
        with open(f'images/{abbreviation}.png', 'wb') as handler:
            handler.write(img_data)  # Save the image to the file
        print(f"Downloaded {abbreviation}.png")
    except Exception as e:
        print(f"Failed to download {abbreviation}: {e}")
print("All images have been downloaded.")

In [ ]:
# Downloading player headshots

df_player_stats = pd.read_csv('data/player_stats.csv')
unique_qbs_with_urls = df_player_stats[df_player_stats['headshot_url'].notna()] \
                        .drop_duplicates(subset=['player_display_name'])
image_folder = 'images'
os.makedirs(image_folder, exist_ok=True)
downloaded_count = 0
total_qbs = len(unique_qbs_with_urls)
for index, row in unique_qbs_with_urls.iterrows():
    player_name = row['player_display_name'].lower().replace(' ', '_')
    headshot_url = row['headshot_url']
    image_path = os.path.join(image_folder, f"{player_name}.png")
    try:
        response = requests.get(headshot_url)
        if response.status_code == 200:
            with open(image_path, 'wb') as file:
                file.write(response.content)
            downloaded_count += 1
            print(f"Downloaded {player_name}'s headshot ({downloaded_count}/{total_qbs}).")
        else:
            print(f"Failed to download {player_name}'s headshot (Status code: {response.status_code}).")
    except Exception as e:
        print(f"Error downloading {player_name}'s headshot: {e}")
print(f"Download complete: {downloaded_count}/{total_qbs} images successfully downloaded.")


# GENERAL STATS

In [3]:
# team1 = 'NYJ'  
# team2 = 'MIN'
# team1 = 'CHI'  # Chicago Bears
# team2 = 'CAR'  # Carolina Panthers
# team1 = 'BAL'  # Baltimore Ravens
# team2 = 'CIN'  # Cincinnati Bengals
# team1 = 'BUF'  # Buffalo Bills
# team2 = 'HOU'  # Houston Texans
# team1 = 'IND'  # Indianapolis Colts
# team2 = 'JAX'  # Jacksonville Jaguars
# team1 = 'MIA'  # Miami Dolphins
# team2 = 'NE'   # New England Patriots
# team1 = 'CLE'  # Cleveland Browns
# team2 = 'WAS'  # Washington Commanders
# team1 = 'LVR'   # Las Vegas Raiders
# team2 = 'DEN'  # Denver Broncos
# team1 = 'SF'   # San Francisco 49ers
# team2 = 'ARI'  # Arizona Cardinals
# team1 = 'GB'   # Green Bay Packers
# team2 = 'LAR'  # Los Angeles Rams
# team1 = 'NYG'  # New York Giants
# team2 = 'SEA'  # Seattle Seahawks
team1 = 'DAL'  # Dallas Cowboys
team2 = 'PIT'  # Pittsburgh Steelers

In [4]:
# Matchup Overview 

games_df = pd.read_csv("data/Games.csv"  )
player_stats_df = pd.read_csv('data/PlayerStats.csv')
conn = sqlite3.connect('data/nfl.db')
cursor = conn.cursor()

records = {}
teams = [team1, team2]
season = 2024
for team in teams:
    query = f"""
    SELECT game_id, week, home_team, away_team, home_score, away_score, home_spread, away_spread
    FROM Games
    WHERE (home_team = '{team}' OR away_team = '{team}') 
      AND season = {season} 
      AND game_type = 'REG';
    """
    cursor.execute(query)
    games = cursor.fetchall()
    ats_wins = 0
    ats_losses = 0
    ats_pushes = 0
    wins = 0
    losses = 0
    for game in games:
        game_id, week, home_team, away_team, home_score, away_score, home_spread, away_spread = game
        if home_score is None and away_score is None:
            continue
        if home_score is not None and away_score is not None:
            spread_diff = home_score - away_score
            try:
                home_spread_float = float(home_spread)
                spread_result = spread_diff + home_spread_float
                if spread_result > 0:
                    if team == home_team:
                        ats_wins += 1
                    else:
                        ats_losses += 1
                elif spread_result < 0:
                    if team == home_team:
                        ats_losses += 1
                    else:
                        ats_wins += 1
                else:
                    ats_pushes += 1
            except (ValueError, TypeError):
                pass
            if home_score > away_score:
                if team == home_team:
                    wins += 1
                else:  
                    losses += 1
            elif home_score < away_score:
                if team == home_team:
                    losses += 1
                else:  
                    wins += 1
    records[team] = {
        'ATS Wins': ats_wins, 
        'ATS Losses': ats_losses,
        'ATS Pushes': ats_pushes,
        'Wins': wins,
        'Losses': losses
    }
conn.close()

result_data = {}
for team in teams:
    team_df = games_df[(games_df['home_team'] == team) | (games_df['away_team'] == team)]
    team_2024_df = team_df[team_df['season'] == 2024]
    team_2024_played_df = team_2024_df[(team_2024_df['home_score'].notnull()) & (team_2024_df['away_score'].notnull())]
    home_points = team_2024_played_df[team_2024_played_df['home_team'] == team]['home_score'].sum()
    away_points = team_2024_played_df[team_2024_played_df['away_team'] == team]['away_score'].sum()
    total_games = team_2024_played_df[(team_2024_played_df['home_team'] == team) | (team_2024_played_df['away_team'] == team)].shape[0]
    points_per_game = (home_points + away_points) / total_games if total_games > 0 else 0
    home_allowed = team_2024_played_df[team_2024_played_df['home_team'] == team]['away_score'].sum()
    away_allowed = team_2024_played_df[team_2024_played_df['away_team'] == team]['home_score'].sum()
    points_allowed_per_game = (home_allowed + away_allowed) / total_games if total_games > 0 else 0
    result_data[f"{team} (2024)"] = {
        "W/L": f"{records[team]['Wins']}W - {records[team]['Losses']}L",
        "ATS": f"{records[team]['ATS Wins']}W - {records[team]['ATS Losses']}L - {records[team]['ATS Pushes']}P",
        "Points Per Game": round(points_per_game, 2),
        "Defensive Points Allowed": round(points_allowed_per_game, 2)
    }
result_df = pd.DataFrame(result_data)
print(result_df)
print('')

# Matchup Stats
team_matchups = games_df[((games_df['home_team'] == team1) & (games_df['away_team'] == team2)) | 
                         ((games_df['home_team'] == team2) & (games_df['away_team'] == team1))]
last_10_games = team_matchups.sort_values(by='date', ascending=False).head(10)
if last_10_games.empty:
    print(f"No recent games found between {team1} and {team2}.")
else:
    total_points = last_10_games['home_score'] + last_10_games['away_score']
    average_total_points = total_points.mean()
    team1_wins = sum((last_10_games['home_team'] == team1) & (last_10_games['home_score'] > last_10_games['away_score'])) + \
                 sum((last_10_games['away_team'] == team1) & (last_10_games['away_score'] > last_10_games['home_score']))
    team2_wins = sum((last_10_games['home_team'] == team2) & (last_10_games['home_score'] > last_10_games['away_score'])) + \
                 sum((last_10_games['away_team'] == team2) & (last_10_games['away_score'] > last_10_games['home_score']))
    team1_scores = last_10_games.loc[last_10_games['home_team'] == team1, 'home_score'].sum() + \
                   last_10_games.loc[last_10_games['away_team'] == team1, 'away_score'].sum()
    team2_scores = last_10_games.loc[last_10_games['home_team'] == team2, 'home_score'].sum() + \
                   last_10_games.loc[last_10_games['away_team'] == team2, 'away_score'].sum()
    team1_home_wins = sum((last_10_games['home_team'] == team1) & (last_10_games['home_score'] > last_10_games['away_score']))
    team1_home_losses = sum((last_10_games['home_team'] == team1) & (last_10_games['home_score'] < last_10_games['away_score']))
    team1_away_wins = sum((last_10_games['away_team'] == team1) & (last_10_games['away_score'] > last_10_games['home_score']))
    team1_away_losses = sum((last_10_games['away_team'] == team1) & (last_10_games['away_score'] < last_10_games['home_score']))
    team2_home_wins = sum((last_10_games['home_team'] == team2) & (last_10_games['home_score'] > last_10_games['away_score']))
    team2_home_losses = sum((last_10_games['home_team'] == team2) & (last_10_games['home_score'] < last_10_games['away_score']))
    team2_away_wins = sum((last_10_games['away_team'] == team2) & (last_10_games['away_score'] > last_10_games['home_score']))
    team2_away_losses = sum((last_10_games['away_team'] == team2) & (last_10_games['away_score'] < last_10_games['home_score']))
    over_50_points_games = sum(total_points > 50)
    print(f"Matchup: {team1} vs {team2}")
    print(f"Total games analyzed between these two teams: {len(last_10_games)}")
    print(f"{team1} Wins: {team1_wins}")
    print(f"{team2} Wins: {team2_wins}")
    print(f"Average Total Points: {average_total_points}")
    print(f"Games with more than 50 total points: {over_50_points_games}")
    print(f"Average points scored by {team1} per game: {team1_scores / len(last_10_games):.2f}")
    print(f"Average points scored by {team2} per game: {team2_scores / len(last_10_games):.2f}")
    print(f"{team1} home record: {team1_home_wins}W {team1_home_losses}L")
    print(f"{team1} away record: {team1_away_wins}W {team1_away_losses}L")
    print(f"{team2} home record: {team2_home_wins}W {team2_home_losses}L")
    print(f"{team2} away record: {team2_away_wins}W {team2_away_losses}L")
    # Print the last 5 games between the teams in a table with full rows
    last_5_games = team_matchups.sort_values(by='date', ascending=False).head(5)
    if last_5_games.empty:
        print(f"No recent games found between {team1} and {team2}.")
    else:
        print("\nLast 5 games between the teams:")
        # Reset index to make sure we don't have any issues with duplicate indices
        last_5_games = last_5_games.reset_index(drop=True)
        # Display all columns without specifying them
        print(last_5_games.to_string(index=False))
    print()

# After calculating result_data and before the final print statements

# Save to Markdown
with open('result_data.md', 'w') as f:
    f.write('# Matchup Overview\n\n')
    
    for team, stats in result_data.items():
        f.write(f"## {team}\n")
        f.write(f"- **W/L**: {stats['W/L']}\n")
        f.write(f"- **ATS**: {stats['ATS']}\n")
        f.write(f"- **Points Per Game**: {stats['Points Per Game']}\n")
        f.write(f"- **Defensive Points Allowed**: {stats['Defensive Points Allowed']}\n")
        f.write('\n')

    f.write('# Matchup Stats\n\n')
    if last_10_games.empty:
        f.write(f"No recent games found between {team1} and {team2}.\n")
    else:
        f.write(f"### Matchup: {team1} vs {team2}\n")
        f.write(f"- **Total games analyzed**: {len(last_10_games)}\n")
        f.write(f"- **{team1} Wins**: {team1_wins}\n")
        f.write(f"- **{team2} Wins**: {team2_wins}\n")
        f.write(f"- **Average Total Points**: {average_total_points:.2f}\n")
        f.write(f"- **Games with more than 50 total points**: {over_50_points_games}\n")
        f.write(f"- **Average points scored by {team1} per game**: {team1_scores / len(last_10_games):.2f}\n")
        f.write(f"- **Average points scored by {team2} per game**: {team2_scores / len(last_10_games):.2f}\n")
        
        f.write('\n### Last 5 games between the teams\n\n')
        f.write('| ' + ' | '.join(last_5_games.columns) + ' |\n')
        f.write('|' + '|'.join(['---' for _ in last_5_games.columns]) + '|\n')
        for _, row in last_5_games.iterrows():
            f.write('| ' + ' | '.join(str(value) for value in row) + ' |\n')


                            DAL (2024)    PIT (2024)
W/L                            2W - 2L       3W - 1L
ATS                       1W - 3L - 0P  3W - 1L - 0P
Points Per Game                  24.25         18.75
Defensive Points Allowed          26.0         13.25

Matchup: DAL vs PIT
Total games analyzed between these two teams: 5
DAL Wins: 2
PIT Wins: 3
Average Total Points: 47.2
Games with more than 50 total points: 2
Average points scored by DAL per game: 22.80
Average points scored by PIT per game: 24.40
DAL home record: 1W 2L
DAL away record: 1W 1L
PIT home record: 1W 1L
PIT away record: 2W 1L

Last 5 games between the teams:
        game_id  season  week game_type                date weekday gametime away_team  away_score home_team  home_score location  result  total  overtime  spread_line  total_line  away_rest  home_rest     roof    surface  temp  wind away_qb_id home_qb_id       away_qb_name       home_qb_name    away_coach    home_coach         referee stadium_id         sta

In [5]:
# # Get stats for a game

# conn = sqlite3.connect('data/nfl.db')

# # Hardcode the Game ID
# game_id = '2023_05_KC_MIN'

# # Fetch general game details from the Games table
# query = f"""
# SELECT game_type, week, date AS gameday, weekday, gametime, away_team, away_score, home_team, home_score, location, spread_line, total_line
# FROM Games
# WHERE game_id = '{game_id}';
# """
# cursor = conn.cursor()
# cursor.execute(query)
# game_details = cursor.fetchall()[0]
# game_columns = [col[0] for col in cursor.description]
# game_details_df = pd.DataFrame([game_details], columns=game_columns)

# # Display general game details using pandas DataFrame
# print(f"\nGeneral Details for Game ID: {game_id}")
# display(game_details_df)  # Use display to render the DataFrame nicely

# # Fetch and display player stats from PlayerStats table
# query = f"SELECT * FROM PlayerStats WHERE game_id = '{game_id}';"
# cursor.execute(query)
# player_stats = cursor.fetchall()
# player_columns = [col[0] for col in cursor.description]
# player_stats_df = pd.DataFrame(player_stats, columns=player_columns)

# # Filter and display stats by position

# # Quarterback stats
# qb_stats_df = player_stats_df[player_stats_df['position'] == 'QB']
# print("\nQuarterback Stats:")
# display(qb_stats_df)

# # Running Back stats
# rb_stats_df = player_stats_df[player_stats_df['position'] == 'RB']
# print("\nRunning Back Stats:")
# display(rb_stats_df)

# # Wide Receiver stats
# wr_stats_df = player_stats_df[player_stats_df['position'] == 'WR']
# print("\nWide Receiver Stats:")
# display(wr_stats_df)

# # Tight End stats
# te_stats_df = player_stats_df[player_stats_df['position'] == 'TE']
# print("\nTight End Stats:")
# display(te_stats_df)

# # Close the connection
# conn.close()



In [6]:
# Analyze player performance this year/recently 

games_df = pd.read_csv('data/Games.csv')
player_stats_df = pd.read_csv('data/PlayerStats.csv')
positions = ['QB', 'WR', 'RB', 'TE']
current_players_team1 = player_stats_df[(player_stats_df['player_current_team'] == team1) & 
                                        (player_stats_df['season'] == 2024)]
current_players_team2 = player_stats_df[(player_stats_df['player_current_team'] == team2) & 
                                        (player_stats_df['season'] == 2024)]

# Open the markdown file in append mode
with open('result_data.md', 'a') as f:
    f.write("\n\n# Player Analysis\n")
    
    for team, current_players in [(team1, current_players_team1), (team2, current_players_team2)]:
        f.write(f"\n## Current 2024 Players for {team}:\n")
        for position in positions:
            players_df = current_players[current_players['position'] == position]
            filtered_players = players_df
            players = filtered_players['player_display_name'].unique()
            f.write(f"### {position}:\n")
            f.write(", ".join(players) + "\n")
        f.write("\n")

    for team, current_players in [(team1, current_players_team1), (team2, current_players_team2)]:
        team_stats = []
        for player in current_players['player_display_name'].unique():
            player_stats = current_players[current_players['player_display_name'] == player]
            if player_stats['position'].isin(['WR', 'TE', 'RB']).all() and player_stats['receptions'].sum() > 0:
                avg_yards_per_reception = player_stats['receiving_yards'].sum() / player_stats['receptions'].sum()
                avg_receptions_per_game = player_stats['receptions'].sum() / player_stats['game_id'].nunique() if player_stats['game_id'].nunique() > 0 else 0
                team_stats.append({
                    'Player': player,
                    'Avg Yards/Reception': round(avg_yards_per_reception, 2),
                    'Avg Receptions/Game': round(avg_receptions_per_game, 2)
                })
        team_df = pd.DataFrame(team_stats)
        team_df = team_df.sort_values(by=['Avg Yards/Reception', 'Avg Receptions/Game'], ascending=False)
        f.write(f"\n## Average Stats for {team} Players (Sorted from Highest to Lowest):\n")
        f.write(team_df.to_markdown(index=False))
        f.write("\n")

# Print confirmation
print("Player analysis has been added to Analysis/result_data.md")

Player analysis has been added to Analysis/result_data.md


In [7]:
# RB and QB Stats & Matchup Analysis

games_df = pd.read_csv('data/Games.csv')
player_stats_df = pd.read_csv('data/PlayerStats.csv')
all_stats_df = pd.read_csv('data/all_passing_rushing_receiving.csv')

positions = ['QB', 'RB']
current_players_team1 = player_stats_df[
    (player_stats_df['player_current_team'] == team1) & 
    (player_stats_df['season'] == 2024)
].groupby(['player_display_name', 'position']).agg({
    'passing_yards': 'sum',
    'rushing_yards': 'sum'
}).reset_index()

current_players_team2 = player_stats_df[
    (player_stats_df['player_current_team'] == team2) & 
    (player_stats_df['season'] == 2024)
].groupby(['player_display_name', 'position']).agg({
    'passing_yards': 'sum',
    'rushing_yards': 'sum'
}).reset_index()

players_team1 = []
players_team2 = []

with open('result_data.md', 'a') as f:
    f.write("\n\n# RB and QB Stats & Matchup Analysis\n")

    for team, current_players, players_list in [(team1, current_players_team1, players_team1), (team2, current_players_team2, players_team2)]:
        f.write(f"\n## Current 2024 Quarterbacks and Running Backs for {team}:\n")
        for position in positions:
            players_df = current_players[current_players['position'] == position]
            if position == 'QB':
                filtered_players = players_df[players_df['passing_yards'] > 100]
            else:  # RB
                filtered_players = players_df[players_df['rushing_yards'] > 50]
            players = filtered_players['player_display_name'].unique()
            players_list.extend(players)
            f.write(f"### {position}:\n")
            f.write(", ".join(players) + "\n")
        f.write("\n")

    # Analyze players from both teams
    for team, players in [(team1, players_team1), (team2, players_team2)]:
        f.write(f"\n## Analyzing players for {team}:\n")
        for player_name in players:
            player_data = all_stats_df[all_stats_df['player'] == player_name]
            
            if player_data.empty:
                f.write(f"No data available for {player_name}\n")
                continue
            
            total_games = player_data.shape[0]
            opponent_team = team2 if team == team1 else team1
            opponent_data = player_data[player_data['opponent_team'] == opponent_team].drop_duplicates(subset=['game_id'])
            
            f.write(f"\n### {player_name}:\n")
            f.write("#### CAREER INSIGHTS:\n")
            f.write(f"- Total Games Played: {total_games}\n")
            f.write(f"- Career Rush Yards: {player_data['rush_yds'].sum()}\n")
            f.write(f"- Career Rush TDs: {player_data['rush_td'].sum()}\n")
            
            f.write("\n#### OPPONENT INSIGHTS:\n")
            if opponent_data.empty:
                f.write(f"No data available for {player_name} against {opponent_team}\n")
            else:
                f.write(f"- Opponent: {opponent_team}\n")
                f.write(f"- Total Games vs Opponent: {opponent_data.shape[0]}\n")
                f.write(f"- Average Rush Yards per Game vs Opponent: {opponent_data['rush_yds'].mean():.2f}\n")
                f.write(f"- Rushing Touchdowns vs Opponent: {opponent_data['rush_td'].sum()}\n")
                yards_per_carry = opponent_data['rush_yds'].sum() / opponent_data['rush_att'].sum() if opponent_data['rush_att'].sum() > 0 else 0
                f.write(f"- Average Yards per Carry vs Opponent: {yards_per_carry:.2f}\n")
                
                f.write("\n#### GAMES AGAINST OPPONENT:\n")
                for index, row in opponent_data.iterrows():
                    f.write(f"{index + 1}. Game ID: {row['game_id']}, Rush Yards: {row['rush_yds']}\n")
            
            f.write("\n---\n")

print("RB and QB Stats & Matchup Analysis has been added to result_data.md")

RB and QB Stats & Matchup Analysis has been added to result_data.md


In [8]:
# WR and TE Stats - Longest Reception & Matchup Stats

games_df = pd.read_csv('data/Games.csv')
player_stats_df = pd.read_csv('data/PlayerStats.csv')
all_stats_df = pd.read_csv('data/all_passing_rushing_receiving.csv')

positions = ['WR', 'TE']
current_players_team1 = player_stats_df[
    (player_stats_df['player_current_team'] == team1) & 
    (player_stats_df['season'] == 2024)
].groupby(['player_display_name', 'position'])['receptions'].sum().reset_index()
current_players_team1 = current_players_team1[current_players_team1['receptions'] >= 3]
current_players_team2 = player_stats_df[
    (player_stats_df['player_current_team'] == team2) & 
    (player_stats_df['season'] == 2024)
].groupby(['player_display_name', 'position'])['receptions'].sum().reset_index()
current_players_team2 = current_players_team2[current_players_team2['receptions'] >= 3]

def get_player_longest_reception_stats(player_name, opponent_team=None):
    player_data = all_stats_df[all_stats_df['player'] == player_name]

    if 'rec_long' not in player_data.columns:
        return f"No reception data available for {player_name}"

    longest_reception = player_data['rec_long'].max()
    total_games = player_data.shape[0]
    opponent_insights = None
    opponent_data = None
    if opponent_team:
        opponent_data = player_data[player_data['opponent_team'] == opponent_team].drop_duplicates(subset=['game_id', 'rec_yds'])
        
        if opponent_data.empty:
            opponent_insights = f"No data available for {player_name} against {opponent_team}"
        else:
            opponent_insights = {
                "Opponent": opponent_team,
                "Longest Reception vs Opponent": opponent_data['rec_long'].max(),
                "Average Longest Reception vs Opponent": opponent_data['rec_long'].mean(),
                "Total Games vs Opponent": opponent_data.shape[0],
                "Games with 30+ Yard Reception vs Opponent": opponent_data[opponent_data['rec_long'] >= 30].shape[0],
                "Average Receptions per Game vs Opponent": opponent_data['rec'].mean(),
                "Average Receiving Yards per Game vs Opponent": opponent_data['rec_yds'].mean(),
                "Receiving Touchdowns vs Opponent": opponent_data['rec_td'].sum(),
                "Average Targets per Game vs Opponent": opponent_data['targets'].mean() if 'targets' in opponent_data.columns else "N/A",
            }
    else:
        opponent_insights = "No opponent provided."

    career_insights = {
        "Player": player_name,
        "Career Longest Reception": longest_reception,
        "Total Games Played": total_games,
    }

    return career_insights, opponent_insights, opponent_data

with open('result_data.md', 'a') as f:
    f.write("\n# WR and TE Analysis\n\n")
    analyzed_players = set()
    for team, players in [(team1, current_players_team1), (team2, current_players_team2)]:
        f.write(f"\n## Analyzing players for {team}:\n")
        for _, player in players[players['position'].isin(['WR', 'TE'])].iterrows():
            player_name = player['player_display_name']
            if player_name not in analyzed_players:
                analyzed_players.add(player_name)
                career_insights, opponent_insights, opponent_data = get_player_longest_reception_stats(player_name, team2 if team == team1 else team1)
                
                f.write(f"\n### {player_name}:\n")
                f.write("#### CAREER INSIGHTS:\n")
                for key, value in career_insights.items():
                    f.write(f"- {key}: {value}\n")
                f.write("\n#### OPPONENT INSIGHTS:\n")
                if isinstance(opponent_insights, dict):
                    for key, value in opponent_insights.items():
                        f.write(f"- {key}: {value}\n")
                else:
                    f.write(f"{opponent_insights}\n")
                if opponent_data is not None and not opponent_data.empty:
                    f.write("\n#### GAMES AGAINST OPPONENT:\n")
                    for index, row in opponent_data.iterrows():
                        f.write(f"{index + 1}. Game ID: {row['game_id']}, Rec Yards: {row['rec_yds']}\n")
                f.write("\n---\n")

print("WR and TE Stats & Matchup Analysis has been added to result_data.md")

WR and TE Stats & Matchup Analysis has been added to result_data.md


In [9]:
# # Function to analyze player performance between two teams

# games_df = pd.read_csv('data/Games.csv')
# player_stats_df = pd.read_csv('data/PlayerStats.csv')

# # Function to identify current 2024 players for each team and find their historical stats in the matchup
# def analyze_current_2024_players_and_their_historical_stats(team1, team2, games_df, player_stats_df):
#     # Step 1: Identify current players from 2024 season for both teams
#     positions = ['WR', 'TE', 'RB', 'QB']  # Optionally filter by positions if needed

#     # Current 2024 players for team1
#     current_players_team1 = player_stats_df[(player_stats_df['player_current_team'] == team1) & 
#                                             (player_stats_df['season'] == 2024)]

#     # Current 2024 players for team2
#     current_players_team2 = player_stats_df[(player_stats_df['player_current_team'] == team2) & 
#                                             (player_stats_df['season'] == 2024)]
    
#     # Get unique list of players' names for each team
#     players_team1_names = current_players_team1['player_display_name'].unique()
#     players_team2_names = current_players_team2['player_display_name'].unique()

#     print(f"\nCurrent 2024 Players for {team1}:")
#     print(players_team1_names)
    
#     print(f"\nCurrent 2024 Players for {team2}:")
#     print(players_team2_names)

#     # Step 2: Find historical stats for these players in any past matchups between the two teams
#     # Filter games between team1 and team2
#     matchup_games = games_df[((games_df['home_team'] == team1) & (games_df['away_team'] == team2)) |
#                              ((games_df['home_team'] == team2) & (games_df['away_team'] == team1))]

#     print("\nHistorical Stats of Current 2024 Players for the Matchup:")
    
#     # Step 3: Retrieve historical stats for players of team1
#     historical_stats_team1 = player_stats_df[(player_stats_df['player_display_name'].isin(players_team1_names)) &
#                                              ((player_stats_df['home_team'] == team1) & (player_stats_df['away_team'] == team2) |
#                                               (player_stats_df['home_team'] == team2) & (player_stats_df['away_team'] == team1))]
    
#     # Step 4: Retrieve historical stats for players of team2
#     historical_stats_team2 = player_stats_df[(player_stats_df['player_display_name'].isin(players_team2_names)) &
#                                              ((player_stats_df['home_team'] == team1) & (player_stats_df['away_team'] == team2) |
#                                               (player_stats_df['home_team'] == team2) & (player_stats_df['away_team'] == team1))]

#     # Step 5: Aggregate the stats and calculate per-game averages for each player
#     def summarize_player_stats(historical_stats):
#         if not historical_stats.empty:
#             # Group by player and calculate averages
#             summary = historical_stats.groupby('player_display_name').agg({
#                 'receptions': 'mean',
#                 'targets': 'mean',
#                 'receiving_yards': 'mean',
#                 'receiving_tds': 'mean',
#                 'carries': 'mean',
#                 'rushing_yards': 'mean',
#                 'rushing_tds': 'mean',
#                 'passing_yards': 'mean',
#                 'passing_tds': 'mean',
#                 'interceptions': 'mean',
#                 'fantasy_points_ppr': 'mean'
#             }).reset_index()
            
#             # Add the number of games played by each player
#             summary['games_played'] = historical_stats.groupby('player_display_name')['game_id'].nunique().values
            
#             return summary
#         else:
#             return pd.DataFrame()

#     # Summarize the stats for both teams
#     summary_team1 = summarize_player_stats(historical_stats_team1)
#     summary_team2 = summarize_player_stats(historical_stats_team2)

#     # Display results for both teams
#     print(f"\nAverage Historical Stats per Game for {team1} players vs {team2}:")
#     if not summary_team1.empty:
#         display(summary_team1)
#     else:
#         print(f"No historical stats found for {team1} players vs {team2}.")
    
#     print(f"\nAverage Historical Stats per Game for {team2} players vs {team1}:")
#     if not summary_team2.empty:
#         display(summary_team2)
#     else:
#         print(f"No historical stats found for {team2} players vs {team1}.")
    
# analyze_current_2024_players_and_their_historical_stats(team1, team2, games_df, player_stats_df)